In [28]:
# Cleaning
import pandas as pd
import os

# import all csv's from a folder and combine them into one dataframe
def combine_data_files(folder_path):
    df_list = []

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            df = pd.read_csv(file_path)
            df_list.append(df)

    return pd.concat(df_list, axis=0, ignore_index=True)

# Once data is imported from csv to a dataframe, clean up the datatypes
def clean_df_new(df):
    
    resMap = lambda res : 'White' if res == '1-0' else 'Black' if res == '0-1' else 'Draw' if res == "1/2-1/2" else res
    cleanDateTime = lambda d: pd.to_datetime(d)
    cleanComment = lambda x: x if x.find(']') == -1 else x.split("] ")[1]

    datetime_columns = ['Date', 'Time', 'WhiteClock', 'BlackClock']
    float_columns = ['WhiteRD', 'BlackRD', 'AvgEvalOpening', 'AvgEvalMiddle', 'AvgEvalEnd', 'AvgEmtOpening', 'AvgEmtMiddle', 'AvgEmtEnd']
    int_columns = ['FICSGamesDBGameNo', 'WhiteElo', 'BlackElo', 'PlyCount']

    df = df.drop(['Event', 'Site', 'Round', 'WhiteClock', 'BlackClock'], axis=1)

    for column in df.columns:

        if column in float_columns:
            df[column] = df[column].astype(float)
        elif column in int_columns:
            df[column] = df[column].astype(int)
        elif column in datetime_columns:
            df[column] = df[column].apply(cleanDateTime)
        else:
            if column == 'Result':
                df[column] = df[column].apply(resMap)
            if column == 'ResultComment':
                df[column] = df[column].apply(cleanComment)
            df[column] = df[column].astype('category')

    df['WhiteIsComp'] = df['WhiteIsComp'] == 'Yes'
    df['BlackIsComp'] = df['BlackIsComp'] == 'Yes'

    return df

In [30]:
combined_df = combine_data_files('data-by-month')
cleaned = clean_df_new(combined_df)
print (cleaned.dtypes)
print(cleaned.shape)
cleaned.head()

Date                 datetime64[ns]
White                      category
Black                      category
Result                     category
BlackElo                      int64
BlackIsComp                    bool
BlackRD                     float64
ECO                        category
FICSGamesDBGameNo             int64
PlyCount                      int64
Time                 datetime64[ns]
TimeControl                category
WhiteElo                      int64
WhiteIsComp                    bool
WhiteRD                     float64
AvgEvalOpening              float64
AvgEvalMiddle               float64
AvgEvalEnd                  float64
AvgEmtOpening               float64
AvgEmtMiddle                float64
AvgEmtEnd                   float64
ResultComment              category
dtype: object
(26032, 22)


,Date,White,Black,Result,BlackElo,BlackIsComp,BlackRD,ECO,FICSGamesDBGameNo,PlyCount,...,WhiteElo,WhiteIsComp,WhiteRD,AvgEvalOpening,AvgEvalMiddle,AvgEvalEnd,AvgEmtOpening,AvgEmtMiddle,AvgEmtEnd,ResultComment
0,2023-02-28,ArasanX,Notarious,Draw,2874,True,37.2,B10,530376668,39,...,2780,True,34.7,15.571429,-7.461538,5.416667,0.126308,3.120000,3.252667,Game drawn by mutual agreement
1,2023-02-28,Notarious,ArasanX,White,2786,True,34.8,D79,530376382,139,...,2868,True,37.4,30.340426,46.913043,104.086957,2.313957,3.352826,0.753348,Black resigns
2,2023-02-28,exeComp,ArasanX,Black,2780,True,34.9,A17,530375680,274,...,2711,True,37.9,59.739130,58.186813,-216.551724,3.552725,0.891088,0.430033,White resigns
3,2023-02-28,nthmaster,Speyside,White,2202,True,45.1,D27,530375325,169,...,2135,False,40.3,201.561404,500.696429,991.975610,2.761375,1.995554,0.277589,Black checkmated
4,2023-02-28,GriffyJr,family,Black,2019,False,48.5,C01,530374794,162,...,2028,True,37.3,7.000000,-108.166667,-410.115385,0.609074,0.589000,0.436094,White resigns


In [31]:
def construct_df_train(df):
    # Define columns to take the average of and to one-hot encode
    cols_to_avg = ['WhiteRD', 'BlackRD', 'AvgEvalOpening', 'AvgEvalMiddle', 'AvgEvalEnd', 'AvgEmtOpening', 'AvgEmtMiddle', 'AvgEmtEnd', 'PlyCount']
    cols_to_onehot = ['ECO'] # 'White', 'Black',  <--- For if we decide to train using player UN for black and white as a features
    ground_truth_labels = ['Result', 'ResultComment']

    # Define dictionary to map column names to letters for one-hot encoding
    col_name_to_letter = {col_name: chr(65+i) for i, col_name in enumerate(ground_truth_labels+cols_to_onehot)}

    # Define column names for player-wise averages
    new_cols_white = ['WhiteAvgRD', 'WhiteAvgEvalForOpenings', 'WhiteAvgEvalForMiddlegames', 'WhiteAvgEvalForEndgames', 'WhiteAvgEmtForOpenings', 'WhiteAvgEmtForMiddlegames', 'WhiteAvgEmtForEndgames', 'WhiteAvgPlyCount']
    new_cols_black = ['BlackAvgRD', 'BlackAvgEvalForOpenings', 'BlackAvgEvalForMiddlegames', 'BlackAvgEvalForEndgames', 'BlackAvgEmtForOpenings', 'BlackAvgEmtForMiddlegames', 'BlackAvgEmtForEndgames', 'BlackAvgPlyCount']

    # Compute player-wise averages
    dataAvgsByUserWhite = {p: df[df['White'] == p][cols_to_avg+cols_to_onehot].mean(numeric_only=True) for p in df['White'].unique()}
    dataAvgsByUserBlack = {p: df[df['Black'] == p][cols_to_avg+cols_to_onehot].mean(numeric_only=True) for p in df['Black'].unique()}

    # Define empty dataframe to hold the new data
    df_new = pd.DataFrame(columns = ground_truth_labels+cols_to_onehot+new_cols_white+new_cols_black)

    # Iterate over each row in the original dataframe and create a new row in the output dataframe with the necessary data
    for row in df.itertuples():
        white_avg_values = dataAvgsByUserWhite.get(row.White, pd.Series([0]*len(cols_to_avg+cols_to_onehot)))
        black_avg_values = dataAvgsByUserBlack.get(row.Black, pd.Series([0]*len(cols_to_avg+cols_to_onehot)))

        white_new_cols = [white_avg_values[0], white_avg_values[2], white_avg_values[3], white_avg_values[4], white_avg_values[5], white_avg_values[6], white_avg_values[7], white_avg_values[8]]
        black_new_cols = [black_avg_values[1], black_avg_values[2], black_avg_values[3], black_avg_values[4], black_avg_values[5], black_avg_values[6], black_avg_values[7], black_avg_values[8]]

        new_row = tuple(getattr(row, label) for label in ground_truth_labels+cols_to_onehot) + tuple(white_new_cols) + tuple(black_new_cols)

        df_new.loc[len(df_new)] = new_row

    # One-hot encode the ground truth labels and cols_to_onehot
    df_new = pd.get_dummies(df_new, columns = ground_truth_labels+cols_to_onehot, prefix = list(col_name_to_letter.values()))

    # Map the column labels to their corresponding uppercase letters
    df_new.columns = df_new.columns.map(lambda x: col_name_to_letter.get(x, x))

    return df_new

In [32]:
df_train = construct_df_train(cleaned)
print (cleaned.dtypes)
cleaned.head()

Date                 datetime64[ns]
White                      category
Black                      category
Result                     category
BlackElo                      int64
BlackIsComp                    bool
BlackRD                     float64
ECO                        category
FICSGamesDBGameNo             int64
PlyCount                      int64
Time                 datetime64[ns]
TimeControl                category
WhiteElo                      int64
WhiteIsComp                    bool
WhiteRD                     float64
AvgEvalOpening              float64
AvgEvalMiddle               float64
AvgEvalEnd                  float64
AvgEmtOpening               float64
AvgEmtMiddle                float64
AvgEmtEnd                   float64
ResultComment              category
dtype: object


,Date,White,Black,Result,BlackElo,BlackIsComp,BlackRD,ECO,FICSGamesDBGameNo,PlyCount,...,WhiteElo,WhiteIsComp,WhiteRD,AvgEvalOpening,AvgEvalMiddle,AvgEvalEnd,AvgEmtOpening,AvgEmtMiddle,AvgEmtEnd,ResultComment
0,2023-02-28,ArasanX,Notarious,Draw,2874,True,37.2,B10,530376668,39,...,2780,True,34.7,15.571429,-7.461538,5.416667,0.126308,3.120000,3.252667,Game drawn by mutual agreement
1,2023-02-28,Notarious,ArasanX,White,2786,True,34.8,D79,530376382,139,...,2868,True,37.4,30.340426,46.913043,104.086957,2.313957,3.352826,0.753348,Black resigns
2,2023-02-28,exeComp,ArasanX,Black,2780,True,34.9,A17,530375680,274,...,2711,True,37.9,59.739130,58.186813,-216.551724,3.552725,0.891088,0.430033,White resigns
3,2023-02-28,nthmaster,Speyside,White,2202,True,45.1,D27,530375325,169,...,2135,False,40.3,201.561404,500.696429,991.975610,2.761375,1.995554,0.277589,Black checkmated
4,2023-02-28,GriffyJr,family,Black,2019,False,48.5,C01,530374794,162,...,2028,True,37.3,7.000000,-108.166667,-410.115385,0.609074,0.589000,0.436094,White resigns


In [33]:
before = cleaned.shape[0]

dropped = cleaned.dropna()

after = dropped.shape[0]

print ('{} / {}'.format(after,before))
print ('Data Loss : {:.2f} %  \n(Mostly due to NaN eval scores. Try increasing engine computation time if this is too high.)'.format(((before - after)*100)/before))

#cleaned.to_csv('out.csv')

25884 / 26032
Data Loss : 0.57 %  
(Mostly due to NaN eval scores. Try increasing engine computation time if this is too high.)
